In [72]:
import os
import pandas as pd
from Data.database import BancodeDados,BancodeDados_cadastro
from modelo.dizimista import dizimista
from modelo.rua import Rua
import numpy as np

In [56]:
cadastro = BancodeDados_cadastro()
cadastro.criar()
cadastro.inserirCadastro("Suely","Carolina98771139","Suely Aparecida","Comunidade Santa Mônica")

False

In [57]:
comunidade = BancodeDados("Comunidade Santa Mônica")
comunidade.criar()

In [58]:
documentos = os.listdir("DadosAno")

In [59]:
datas = list()
path = "DadosAno/"
for doc in documentos:
    datas.append(pd.read_excel(path+doc))

In [60]:
anos = ["2021","2022","2023"]
meses = ["Janeiro","Fevereiro","Março","Abril","Maio","Junho","Julho","Agosto","Setembro","Outubro","Novembro","Dezembro"]

### Arrumar uma forma de pegar os dados dos dizimistas e inserir no banco de dados 
A dica é usar os anos e meses para preencher as doações, pegar os dizimistas

In [61]:
datas[0]

,Unnamed: 0,Dizimista,N° Casa,Aniversário,Janeiro,Fevereiro,Março,Abril,Maio,Junho,Julho,Agosto,Setembro,Outubro,Novembro,Dezembro,Rua,Zelador
0,0,Renilde das Graças,758,20/07/2021,0,0,0,0,0,0,0,0,0,0,0,0,Palmeiras,Leire
1,1,Mirian da Conceição Neves,769A,30/11/1957,0,0,0,0,0,0,1,1,0,1,1,1,Palmeiras,Leire
2,2,Nilton Antônio Neves,769A,08/08/2021,0,0,0,0,0,0,1,1,1,1,1,1,Palmeiras,Leire
3,3,Julia Alves,823,20/01/2021,0,0,0,0,0,0,0,1,1,1,0,0,Palmeiras,Leire
4,4,Wiliam José,823,09/09/2021,0,0,0,0,0,0,0,1,1,1,0,0,Palmeiras,Leire
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243,243,Vilma Layses da Rocha,205,12/10/1800,0,0,0,0,0,0,0,0,0,0,1,0,Madressilvas,Ivone
244,244,Gabriel Henrique da Costa Oliveira,35,12/10/1800,0,0,0,0,0,0,0,0,0,0,1,0,Aurelio Lopes,Maria da Conceição
245,245,Elaine Cristina Ferreira da Costa,35,12/10/1800,0,0,0,0,0,0,0,0,0,0,1,0,Aurelio Lopes,Maria da Conceição
246,246,Josilany da Consolação,721,12/10/1800,0,0,0,0,0,0,0,0,0,0,0,1,Papoulas,Valter


In [62]:
# exemplo de acesso a linha
datas[0].loc[0]

Unnamed: 0                      0
Dizimista      Renilde das Graças
N° Casa                       758
Aniversário            20/07/2021
Janeiro                         0
Fevereiro                       0
Março                           0
Abril                           0
Maio                            0
Junho                           0
Julho                           0
Agosto                          0
Setembro                        0
Outubro                         0
Novembro                        0
Dezembro                        0
Rua                     Palmeiras
Zelador                     Leire
Name: 0, dtype: object

In [63]:
datas[0]['Aniversário'][0][:2] #dia do aniversário

'20'

In [64]:
datas[0]['Aniversário'][0][3:5] #mes do aniversário

'07'

In [65]:
def getNiver(string:str):
    if string == "12/10/1800":
        return "/"
    return string[:2] + "/" + string[3:5]

In [66]:
def inserirDataFrame(df:pd.DataFrame,ano:str):
    tam = len(df)
    for i in range(tam):
        linha = df.loc[i]
        comunidade.inserirDizimista(linha["Dizimista"],linha["N° Casa"],getNiver(linha["Aniversário"]),linha["Rua"])
        comunidade.inserirRua(linha['Rua'],linha['Zelador'])
        for mes in meses:
            if linha[mes]:
                comunidade.marcarContribuinte(linha["Dizimista"],linha["Rua"],linha["N° Casa"],mes,ano)

In [67]:
datas[0].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 248 entries, 0 to 247
Data columns (total 18 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   248 non-null    int64 
 1   Dizimista    248 non-null    object
 2   N° Casa      244 non-null    object
 3   Aniversário  248 non-null    object
 4   Janeiro      248 non-null    int64 
 5   Fevereiro    248 non-null    int64 
 6   Março        248 non-null    int64 
 7   Abril        248 non-null    int64 
 8   Maio         248 non-null    int64 
 9   Junho        248 non-null    int64 
 10  Julho        248 non-null    int64 
 11  Agosto       248 non-null    int64 
 12  Setembro     248 non-null    int64 
 13  Outubro      248 non-null    int64 
 14  Novembro     248 non-null    int64 
 15  Dezembro     248 non-null    int64 
 16  Rua          248 non-null    object
 17  Zelador      248 non-null    object
dtypes: int64(13), object(5)
memory usage: 35.0+ KB


In [99]:
def nullTovazio(value)->str:
    if str(value) == "nan":
        return ""
    return str(value)

In [101]:
for ano in anos:
    datas[anos.index(ano)]["N° Casa"] = datas[anos.index(ano)]["N° Casa"].apply(lambda x: nullTovazio(x))
    datas[anos.index(ano)]['Aniversário'] = datas[anos.index(ano)]['Aniversário'].astype(str)
    dataFrame = datas[anos.index(ano)]
    inserirDataFrame(dataFrame,ano)